In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from sklearn import metrics
import time

In [ ]:
from keras.layers import Input, Dense, Convolution2D, \
    MaxPooling2D, UpSampling2D, BatchNormalization, Dropout, \
    UpSampling2D, Layer, Flatten, Activation, Reshape, GRU
from keras.models import Model, Sequential
from keras import backend as K
from keras.layers.advanced_activations import ELU
from keras.callbacks import TensorBoard

In [ ]:
RAND_SEED = 777
np.random.seed(RAND_SEED)

In [ ]:
IMG_SHAPE = (16 * 64,)
N_CH = 16

# axis 0 is batch sample index
AXIS_FREQ = 1
AXIS_TIME = 2
AXIS_CH = 3

VAL_DATA_FILE = 'out/val-msgs.mem'
VAL_YS_FILE = 'out/val-ys.npy'
VAL_PATS_FILE = 'out/val-pats.npy'

N_TRAIN_FILES = 1
TRAIN_DATA_FILE = 'out/train-msgs-%d.mem'
TRAIN_YS_FILE = 'out/train-ys-%d.npy'

MODEL_PATH = 'out/models/e-f%d-e%d-auc%.5f'

In [ ]:
# load validation data
X_val = np.memmap(VAL_DATA_FILE, dtype=np.float32, mode='r+').reshape((-1,) + IMG_SHAPE)

X_val = np.divide(X_val, 255.)

pats_val = np.load(VAL_PATS_FILE)
y_val = np.load(VAL_YS_FILE)
y_val = np.repeat(y_val, N_CH, axis=0)

In [ ]:
scores = []

In [ ]:
def score_auc():
    s = 0
    n = X_val.shape[0]
    
    print '\n'
    y_p = model.predict(X_val[s:s+n], verbose=True, batch_size=32)
    y_p = np.nan_to_num(y_p)
    return metrics.roc_auc_score(y_val[s:s+n].T[0], y_p.T[0])

In [ ]:
class MyCallback(keras.callbacks.Callback):
    def _validate(self, epoch):
        s = score_auc()
        scores.append(s)
        np.save('out/scores.npy', scores)
        print "\n\n AUC = %.5f\n"%(s)
        if True or len(scores) == 0 or s >= max(scores):
            f = MODEL_PATH%(train_file_n, epoch, s)
            print 'Saving to: ', f, '\n'
            model.save(f)
#     def on_train_begin(self, epoch, logs={}):
#         self._validate(0)
    def on_epoch_end(self, epoch, logs={}):
        self._validate(1 + epoch)
    def on_epoch_begin(self, epoch, logs={}):
        print '\n'

In [ ]:
model = Sequential()

model.add(Dense(128, input_shape=IMG_SHAPE, activation='tanh'))

model.add(Dropout(0.5))

model.add(Dense(128, activation='tanh'))

model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

In [ ]:
# model.output_shape

In [ ]:
model.compile(optimizer='SGD', loss='categorical_crossentropy')

In [ ]:
for train_file_n in range(N_TRAIN_FILES):

    X_file = TRAIN_DATA_FILE%(1 + train_file_n)
    y_file = TRAIN_YS_FILE%(1 + train_file_n)
    
    print 'Using file', X_file
    
    X_train = np.fromfile(X_file, dtype=np.float32)
    X_train = X_train.reshape((-1,) + IMG_SHAPE)
    X_train = np.divide(X_train, 255.)
    
    y_train = np.load(y_file)
    y_train = y_train.reshape(-1, 2)
    
    y_train = np.repeat(y_train, N_CH, axis=0)

In [ ]:
X_train_ = []
y_train_ = []

for i in range(len(y_train)):
    if y_train[i][0] == 1. or (np.random.rand() <= 0.16):
        X_train_.append(X_train[i])
        y_train_.append(y_train[i])

In [ ]:
X_train_ = np.array(X_train_, dtype=np.float32)
y_train_ = np.array(y_train_, dtype=np.float32)

In [15]:
model.fit(
    X_train_,
    y_train_,
    batch_size=16,
    nb_epoch=300,
    validation_data=(X_val, y_val),
    shuffle=False,
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            'out/models/j-e{epoch:02d}-l{loss:.5f}.hdf5', monitor='loss', verbose=1,
            save_best_only=False, save_weights_only=False, mode='auto'),
        TensorBoard(log_dir='/tmp/tf-mls-j')
    ]
)

17916/17916 [==============================] - 2s - loss: 0.6302 - val_loss: 0.5433
Epoch 45/300
17916/17916 [==============================] - 2s - loss: 0.6299 - val_loss: 0.5497
Epoch 46/300
17916/17916 [==============================] - 2s - loss: 0.6301 - val_loss: 0.5516
Epoch 47/300
17916/17916 [==============================] - 2s - loss: 0.6292 - val_loss: 0.5509
Epoch 48/300
17916/17916 [==============================] - 2s - loss: 0.6293 - val_loss: 0.5543
Epoch 49/300
17916/17916 [==============================] - 2s - loss: 0.6292 - val_loss: 0.5488
Epoch 50/300
17916/17916 [==============================] - 2s - loss: 0.6292 - val_loss: 0.5509
Epoch 51/300
17916/17916 [==============================] - 2s - loss: 0.6287 - val_loss: 0.5495
Epoch 52/300
17916/17916 [==============================] - 2s - loss: 0.6285 - val_loss: 0.5478
Epoch 53/300
17916/17916 [==============================] - 2s - loss: 0.6285 - val_loss: 0.5490
Epoch 54/300
17916/17916 [=================

KeyboardInterrupt: 

In [ ]:
print y_train.shape
plt.hist(y_train_.T[0], bins=2)

In [ ]:
import matplotlib.cm as cm
plt.imshow(np.flipud(d[0]), aspect=10)

In [ ]:
np.repeat([[1,2],[3,4]], 3, axis=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
score_auc()

In [ ]:
y_p = model.predict(X_val, verbose=True, batch_size=32)

In [ ]:
y_p.T[0]

In [ ]:
plt.hist(y_p.T[0])

In [ ]:
score_auc()